<a href="https://colab.research.google.com/github/lunduniversity/schoolprog-satellite/blob/master/exercises/forest_fires/skogsbrander2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uppskatta storleken på skogsbränder

![Bild på en sjö](https://cdn.pixabay.com/photo/2018/11/25/05/43/forest-fire-3836834_1280.jpg)
<small>[Foto](https://pixabay.com/sv/illustrations/forest-fire-forest-3836834/) från Pixabay / [Pixabay License](https://pixabay.com/sv/service/license/)</small>

Med klimatförändringarna blir vädret mer extremt. Extrem torka och värme gör att skogsbränder kan bli allt vanligare. Sommaren 2018 bröt flera stora bränder ut i Sverige. Det största brandområdet var runt den lilla byn Kårböle i norra Hälsingland.

I denna uppgift ska vi beräkna storleken på en skogsbrand genom att mäta på satellitbilder. Vi kommer att analysera bilder före och efter branden för att se hur skogsytan har minskat.

Om du har gjort uppgiften Torkan A tidigare, så kommer du att känna igen NDVI-metoden som vi använde där för att förstå vad som är vad i en satellitbild.

För att beräkna brandområdets yta kommer vi att ringa in det med hjälp av en *polygon* (en månghörning).

## 1. Polygoner

Vi ska börja med att bekanta oss med hur man kan programmera med polygoner och rita dem i Python.

**Uppdrag 1a** Kör koden nedan som importerar några Python-bibliotek vi kommer att behöva.

Vi importerar dels biblioteken `numpy` och `matplotlib.pyplot` som vi använt tidigare. Vi importerar också bibliotek som används för polygoner.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

print("Klart!")


**Uppdrag 1b** Kör koden nedan som ritar en polygon.

In [ ]:
points = [[2, 1], [8, 2], [6, 5]]
polygon = plt.Polygon(points)
plt.xticks(np.arange(0, 11, 1))
plt.yticks(np.arange(0, 11, 1))
plt.grid()
plt.gca().add_patch(polygon)
plt.show()

Att observera:

* En triangel (trehörning) ritades med hörn i punkterna `[2, 1], [8, 2], [6, 5]`
* Funktionerna `xticks` och `yticks` används för att bestämma hur rutnätet ska se ut (min-värde, max-värde och avstånd).

Vi kan alltså definiera en polygon i Python genom att skapa en lista av hörn. 

**Uppdrag 1c** Kopiera koden till rutan nedan. Modifiera koden så att en fyrhörning ritas ut i stället för en triangel. Var vill du sätta det fjärde hörnet? Spelar det någon roll var i listan du lägger till hörnet?



In [ ]:
# Kopiera koden från 1b och modifiera till en fyrhörning

<details>
<summary markdown="span">
Lösning
</summary>
Ändra t.ex. definitionen av points till:
<pre><code>points = [[2, 1], [8, 2], [6, 5], [3, 7]]
</code></pre>
Polygonen ritas ut med räta linjer mellan hörnen enligt ordningen i listan. Om du lägger hörnen i en annan ordning kan du få konturen att korsa sig själv. Prova t.ex.
<pre><code>points = [[2, 1], [8, 2], [3, 7], [6, 5]]
</code></pre>
</details>
<p></p>

**Uppdrag 1d** Experimentera mer med polygoner i kodrutan nedan. Kan du t.ex. skapa följande simmande fisk? Eller hitta på en annan polygon. Experimentera både med polygonen och rutnätet!

<img src="https://github.com/lunduniversity/schoolprog-satellite/raw/master/exercises/forest_fires/polygon-fish.png" width="400"/>





In [ ]:
# Kopiera koden från 1b och modifiera till en fisk eller annan figur

<details>
<summary markdown="span">
Tips
</summary>
<p>Figuren kan skapas med följande kod. Här har vi också ändrat på färgen både på fisken och dess kontur. Men du kan säkert hitta på en egen figur och ett eget rutnät.
<pre><code>points = [[-80, 0], [-40, 40], [20, -10], [65, 30], [60, -40], [20, -20], [-60, -30]]
polygon = plt.Polygon(points)
polygon.set_facecolor("red")
polygon.set_edgecolor("black")
plt.xticks(np.arange(-100, 100, 20))
plt.yticks(np.arange(-100, 100, 20))
plt.grid()
plt.gca().add_patch(polygon)
plt.show()
</code></pre>
</p>
</details>
<p></p>


För att beräkna arean för en polygon kan vi använda följande funktion.

**Uppdrag 1e** Kör koden nedan så att funktionen definieras (så att vi kan anropa den senare).

In [ ]:
def area_calc(corners):
  area=0.0
  n=len(corners)
  for i in range(n):
    (xi, yi) = corners[i]         # coordinates for the i corner
    (xi1, yi1) = corners[(i+1)%n] # coordinates for the next corner
    area += xi*yi1 - xi1*yi
  area *=0.5
  return abs(area)

print("Klart!")

Funktionen räknar ut arean med hjälp av en listig formel som användes av lantmätare redan på 1700-talet för att mäta arean på jordegendomar.

<details>
<summary markdown="span">
Mer om formeln
</summary>
Iden med formeln är att följa punkterna varvet runt och summera trianglarna som bildas av varje sida i kombination med origo. Vissa triangelareor blir positiva och andra negativa, och summan blir arean av polygonen. I varje steg räknas dubbla triangelarean ut som $x_i*y_{i+1}-x_{i+1}*y_i$ som man kan se i koden ovan. På slutet halverar man värdet för att få rätt resultat. Man tar också absolutvärdet eftersom totalsumman kan bli positiv eller negativ beroende på vilket håll man följer punkterna. Mer information om formeln finns t.ex. i <a href="https://en.wikipedia.org/wiki/Shoelace_formula">denna wikipedia-artikel</a>.
</details>
<p></p>


För säkerhets skull ska vi kontrollera att funktionen verkar fungera.



**Uppdrag 1f** Beräkna arean av en polygon genom att anropa funktionen i kodrutan nedan. Verkar resultatet stämma?

<details>
<summary markdown="span">
Tips!
</summary>
För att se om formeln verkar stämma kan du räkna antalet rutor i din polygon som är fyllda till hälften eller mer. Multiplicera sedan med ytan på en ruta. Då får du ett ungefärligt värde på arean.
</details>
<p></p>


In [ ]:
# Byt ut ??? mot en av polygonerna du använt ovan.
points = ???
area_calc(points)

##2. Analysera en skogsbrand

Nu ska vi använda polygoner för att undersöka och uppskatta storleken på en skogsbrand. Vi har förberett satellitbilder dels för skogsbranden i Kårböle 2018 (uppgift 2A) och dels för skogsbranden på Kangaroo Island i Australien 2019 (uppgift 2B).

Välj vilken av uppgifterna du vill arbeta med (avsnitt 2A eller 2B) - eller gör båda om du har tid!

### 2A. Sverige - Skogsbranden i Kårböle 2018
<small>Klicka på den lilla pilen till vänster om rubriken för att fälla ut eller ihop avsnittet.</small>

2018 var ett år med många kraftiga skogsbränder i norra Sverige. Kårböle vid älven Ljusnan var särskilt drabbat.




**Uppdrag 2A.a** Kör kodblocket nedan för att få fram en karta över området. Om du råkat zooma eller panna så du inte ser området, så prova att köra koden en gång till. Kan du zooma ut och se var i Sverige området ligger?

In [ ]:
#@title Kör detta dolda kodblock för att se var brandområdet ligger
import folium
m = folium.Map(
    location=[62,15.3],
    tiles='Stamen Terrain',
    zoom_start=9
)

folium.Rectangle(
    bounds=[(62.05,15.25), (61.89,15.55)],
    fill=False
).add_to(m)


m

**Uppdrag 2A.b** Kör nedanstående kodblock för att ladda ner två satellitbilder över området. Den ena är en bild från hösten 2016, och den andra hösten 2019.

In [ ]:
!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/forest_fires/Sweden/bands_2019_swe.npz --quiet
!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/forest_fires/Sweden/bands_2016_swe.npz --quiet

#Banden för den svenska skogsbranden
bands16 = np.load('bands_2016_swe.npz')
bands19 = np.load('bands_2019_swe.npz')

#Plockar ut banden var för sig som float
#Färgbanden för hösten 2016
blue16 = bands16["blue"].astype(float)
green16 = bands16["green"].astype(float)
red16 = bands16["red"].astype(float)
nir16 = bands16["nir"].astype(float)

#Färgbanden för hösten 2019
blue19 = bands19["blue"].astype(float)
green19 = bands19["green"].astype(float)
red19 = bands19["red"].astype(float)
nir19 = bands19["nir"].astype(float)

print("Klart!")

Till att börja med kan du undersöka färgbilder av området. För att skapa färgbilderna behöver du kombinera tre färgband som satelliten mäter. En vanlig färgbild är en kombination av rött, grönt och blått ljus.  Du kanske har lagt märke till att det finns ytterligare färgband i koden ovan som kallas `nir16` eller `nir19`. NIR är förkortning för Near InfraRed, vilket är det ljus som har lite längre våglängd än rött. Det går inte att se med blotta ögat men satelliten har instrument som kan plocka upp det. NIR kommer att behövas lite senare i uppgiften.

**Uppdrag 2A.c** Kör kodrutan nedan för att definiera funktionen `color_img` som skapar en färgbild av det röda, gröna och blåa bandet.

In [ ]:
def color_img(red,green,blue):
  rgb=np.zeros([500,500,3],dtype=np.uint8)
  divide=8
  for x in range(500):
    for y in range(500):
      r = min(255, red[x][y]/divide)
      g = min(255, green[x][y]/divide)
      b = min(255, blue[x][y]/divide)
      rgb[x,y] = [r,g,b]
  return rgb

print("Klart!")

Funktionen skapar en bild på ett format som passar för plottning.

**Uppdrag 2A.d** Kör kodrutan nedan för att importera biblioteket PIL som vi ska använda för plottning.

In [ ]:
from PIL import Image

print("Klart!")

Nu ska vi plotta färgbilderna för 2016 och 2019 för att se om det är någon skillnad mellan åren.

**Uppdrag 2A.e** Kör kodrutan nedan för att plotta färgbilden för 2016. Kan du urskilja skog, älv och barmark (t.ex. vägar)?

In [ ]:
rgb=color_img(red16,green16,blue16)

img = Image.fromarray(rgb)
display(img)


**Uppdrag 2A.f** Modifiera koden i rutan nedan så att du i stället plottar färgbilden för 2019. Färgbanden ska vara i ordningen rött, grönt, blått. Ser du någon skillnad mot 2016? Var tror du att branden har härjat?

In [ ]:
# Byt ut ??? så du får färgbilden för 2019.
rgb=color_img(???,???,???)

img = Image.fromarray(rgb)
display(img)

<details>
<summary markdown="span">
Svar
</summary>
För att se färgbilden för hösten 2019:
<pre><code>rgb=color_img(red19,green19,blue19)
</code></pre>
</details>
<p></p>

I nästa steg ska vi skapa en bild som visar Normalized Difference Vegetation Index (NDVI). Förhoppningsvis har du stött på NDVI i tidigare uppgifter, som Torkan A.

Som du kanske kommer ihåg så är NDVI ett tal mellan -1 och 1, och som kan användas för att dela upp en landyta i olika kategorier. Ett index som är över 0.1 kan i de flesta fall sägas vara någon form av vegetation. Ju högre NDVI desto tätare växtlighet. Ett NDVI-värde mellan -0.1 till 0.1 brukar indikera en landyta fri från vegetation. Negativa NDVI-värden mellan -0.1 till -1 brukar istället betyda att det är en yta täckt av vatten.

NDVI räknas ut med hjälp av NIR och det röda bandet enligt följande formel: 

           NIR - Red
    NDVI = ---------
           NIR + Red

NDVI-formeln utnyttjar att reflektionen skiljer sig mycket åt mellan NIR och rött ljus för flera slags ytor. Växtlighet absorberar mycket rött ljus, men reflekterar NIR-ljus i hög grad. Med vatten förhåller det sig tvärtom. 

**Uppgift 2A.g** Skriv en funktion med namn `ndvi_img` som räknar ut NDVI i följande kodblock. Om du är osäker på syntaxen får du ett färdigt kodskelett under tipsfliken. Du behöver dock själv skriva in vilka argument funktionen behöver och hur NDVI ska räknas ut. Om du har gjort funktioner innan så prova gärna själv innan du kikar på tipset. Var noga med placeringen av parenteser!

In [ ]:
#Skriv din funktion ndvi_img här:
def ...


print("Klart!") # Utskriften här är bara för att du ska se att det händer något när kodrutan körs

<details>
<summary markdown="span">
Tips!
</summary>
<p><pre><code>def ndvi_img(???,???):
  #skriv hur ndvi räknas ut här:
  ndvi=
  return ndvi</code></pre>
</p>
</details>
<p></p>

<details>
<summary markdown="span">
Lösning
</summary>
<p><pre><code>def ndvi_img(nir,red):
 ndvi=(nir-red)/(nir+red)
 return ndvi</code></pre>
</p>
</details>
<p></p>

Nu kan du prova din funktion genom att se hur NDVI för området ser ut. Börja med år 2016 och fortsätt sedan med 2019.

**Uppdrag 2A.h** Använd de två kodblocken nedan för att plotta NDVI för 2016 och 2019. För varje kodblock, skriv in rätt argument i anropet av din `ndvi_img` funktion och kör sedan kodblocket. Om du är osäker på namnen på färgbanden så kan du hitta dem i kodblocket som skapar färgbilderna. 

In [ ]:
#Byt ut ??? mot namnen på rätt färgband:
ndvi16 = ndvi_img(???,???)

plt.figure(figsize=(10,10))
plt.pcolormesh(ndvi16, cmap='brg')
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
plt.ylim(ndvi16.shape[0], 0)
plt.clim(-1.0, 0.7)
plt.colorbar(label='NDVI')
plt.show()

In [ ]:
#Byt ut ??? mot namnen på rätt färgband:
ndvi19 = ndvi_img(???,???)

plt.figure(figsize=(10,10))
plt.pcolormesh(ndvi19, cmap='brg')
plt.ylim(ndvi19.shape[0], 0)
plt.clim(-1.0, 0.7)
plt.colorbar(label='NDVI')
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
plt.show()


<details>
<summary markdown="span">
Lösning
</summary>
<pre><code> ndvi16=ndvi_img(nir16,red16)
 ndvi19=ndvi_img(nir19,red19)
</code></pre>
</details>
<p></p>

**Uppdrag 2A.i**
Jämför de båda NDVI-bilderna. Kan du identifiera vad som är skog, vatten och barmark? Är det lättare att se skillnader om man tittar på NDVI jämfört med färgbilder?
I vilka områden tror du skogsbranden har ägt rum?

**Uppdrag 2A.j** 
Scrolla upp och titta på färgbilderna igen, lokalisera var det finns moln. Kan du se molnen även i bilderna med NDVI? Går det att skilja på brandområdena och moln i NDVI-bilden?

<details>
<summary markdown="span">
Kommentar
</summary>
Det går inte så bra att se skillnad på moln och barmark i NDVI-bilden. Det kan därför vara bra att titta både på färgbilderna och NDVI-bilderna för att avgöra var det har brunnit.
</details>
<p></p>


Som du kanske lagt märke till i NDVI-bilden från 2019 så är det främst två områden som är intressanta. Ett precis norr om älven och ett söder om. Vi behöver nu skapa två polygoner för att uppskatta storleken på dessa områden. 

**Uppdrag 2A.k**
Börja med att köra kodblocket nedan för att få upp en större bild av NDVI över området år 2019.



In [ ]:
plt.figure(figsize=(15,15))
plt.pcolormesh(ndvi19, cmap='brg')
plt.ylim(ndvi19.shape[0], 0)
plt.clim(-1.0, 0.7)
plt.colorbar(label='NDVI')
plt.xticks(np.arange(0, 500+1, 25.0))
plt.yticks(np.arange(0, 500+1, 25.0))
plt.grid()
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True 

#Bygg ut listan med fler hörn så att hela norra delen av branden täcks:
corner_north = [[140,45],[175,55],[200,54]]
polygon = plt.Polygon(corner_north,fill=None)
plt.gca().add_patch(polygon)

#Bygg ut listan med fler hörn så att hela södra delen av branden täcks:
corner_south=[[200,260],[275,320],[300,310]]
polygon = plt.Polygon(corner_south,fill=None)
plt.gca().add_patch(polygon)
plt.show()


Om du tittar noga på bilden så ser du två trekantiga polygoner. Hittar du dem i koden också?

**Uppdrag 2A.l** Lägg till 3-4 hörn till i `corner_north` och `corner_south` så att polygonerna täcker områdena ungefärligt. Lägg sedan till fler hörn och anpassa polygonerna så att de täcker brandområdena mer noggrannt. Du väljer själv hur noggrann du vill vara. Kör kodrutan efter varje ändring du gör, så du kontrollerar hur resultatet blir.

<details>
<summary markdown="span">
Tips
</summary>
Du kan behöva runt 15 hörn för att få en riktigt noggrann täckning, som i följande bild. Men det räcker med mycket färre hörn för att få en ungefärlig uppskattning.<br>

<img src="https://github.com/lunduniversity/schoolprog-satellite/raw/master/exercises/forest_fires/SWE_firepolygons.png" width="800"/>
</details>
<p></p>

Nu ska vi räkna ut arean på polygonerna. Vi kan använda funktionen `area_calc` som vi definerade tidigare. Parameter till area_calc är listan av koordinater. Fyll i rätt parametrar nedan och räkna ut totalarean.

Funktionen `area_calc` kan användas för att räkna ut pixelarean till dina två polygoner. Parameter till funktionen är en lista med koordinater för dina polygonkanter.

**Uppdrag 2A.m** Skriv in rätt parametervärde till `area_calc` för att räkna ut arean i kodblocket nedan. Räkna sedan ut totalarean.

 

In [ ]:
#Byt ut ??? mot rätt parametrar
area_north=area_calc(???)
area_south=area_calc(???)

totalarea=area_north+area_south
print(totalarea, 'pixlar')


<details>
<summary markdown="span">
Svar
</summary>
<p><pre><code>area_north=area_calc(corner_north)
area_south=area_calc(corner_south)</code></pre>
Arean borde vara någonstans mellan 38 000 - 58 000 pixlar.
</p>
</details>
<p></p>

Nu har vi räknat ut arean i pixlar. Men hur mycket kan det motsvara i $km^2$ (kvadratkilometer)?

Koordinataxlarna i bilden är graderad i pixlar. En pixel i bilden motsvarar ett område på 40x40 $m$ vilket i sin tur motsvarar 1600 $m^2$ (kvadratmeter). Om du är osäker på omvandlingen kan du börja med att räkna ut hur många $m^2$ brandområdet motsvarar.

**Uppdrag 2A.n** Räkna ut arean i kvadratmeter i kodblocket nedan.



In [ ]:
# Byt ut ??? för att räkna ut vad arean är i kvadratmeter?
area_m2 = ???
print(area_m2,'m^2 (kvadratmeter)')

<details>
<summary markdown="span">
Lösning
</summary>
<p><pre><code> area_m2=totalarea*1600
</code></pre>
</p>
</details>
<p></p>

**Uppdrag 2A.o** Sen behöver du fundera ut hur arean kan omvandlas till $km^2$. Skriv in din omvandling i kodblocket nedan. 

In [ ]:
# Byt ut ??? för att räkna ut arean i kvadratkilometer
area_km2 = ???
print(area_km2, 'km^2 (kvadratkilometer)')


<details>
<summary markdown="span">
Tips!
</summary>
<p>Som du säkert redan vet är 1000 meter samma som 1 km. 1 $km^2$ är då samma som $1000^2 m^2$, alltså $1000000 m^2$.
</p>
</details>
<p></p>

<details>
<summary markdown="span">
Lösning
</summary>
<p><pre><code> area_km2=area_m2/1000000
</code></pre>
</p>
</details>
<p></p>

Skog räknar man ofta i *hektar*, där en hektar är 10000 $m^2$, eller 0.01 $km^2$. Multiplicera alltså ditt resultat med 100 för att få det antal hektar som brann i de två bränderna Kårböle vi tittat på. Ett annat mått på ytor är "fotbollsplaner". En fotbollsplan är ofta 60 x 90 meter, dvs det går ungefär 2 fotbollsplaner på en hektar.

I en studie från MSB där man analyserat bränder i Sverige mellan 1996-2018 fann man att det i genomsnitt brunnit 2900 hektar skogsmark om året i hela Sverige.

**Uppgift 2A.p** Hur många hektar resp. fotbollsplaner brann i Kårböle enligt din beräkning? Räkna gärna ut det genom att programmera i kodrutan nedan. Var det mycket eller lite skog som brann? Vad händer i samhället när det blir stora bränder? Kan vi göra något för att förhindra skogsbränder i framtiden? Diskutera gärna med en kompis.


In [ ]:
# Räkna ut hur många hektar som brann, och hur många fotbollsplaner det motsvarar.


<details>
<summary markdown="span">
Kommentar
</summary>
Det fanns en tredje brand i Kårböle som vi kan skymta uppe till vänster på bilderna. Skogsstyrelsen räknar med att det sammanlagt brann 9000 hektar i Kårböle, och totalt 21000 hektar i hela Sverige sommaren 2018.<br><br>
Skeendet blev dramatiskt med många bränder samtidigt. Befolkningen fick evakueras på flera ställen, inklusive Kårböle, och Sverige fick hjälp med brandmän och vattenbombning från många andra länder, bl.a. Italien, Frankrike, Portugal, Norge, Danmark, Polen, Litauen och Tyskland.<br><br>
Du kan läsa mer om skogsbränderna 2018 i t.ex. <a href="https://sv.wikipedia.org/wiki/Skogsbränderna_i_Sverige_2018">denna wikipedia-artikel</a> och <a href="https://www.svt.se/nyheter/lokalt/gavleborg/sa-har-stora-var-branderna">detta SVT-reportage</a>.
</details>
<p></p>

### 2B. Australien - Skogsbranden på Kangaroo Island 2019
<small>Klicka på den lilla pilen till vänster om rubriken för att fälla ut eller ihop avsnittet.</small>

Årsskiftet 2019-2020 skedde en brandkatastrof i stora delar av Australien. I den här uppgiften ska du få undersöka brandområdet på Kangaroo Island, en ö som ligger längs Australiens södra kust.

**Uppgift 2B.a** Kör kodblocket nedan för att få fram en karta över ön. Om du råkat zooma eller panna så du inte ser ön, så prova att köra koden en gång till. Kan du zooma ut och se var ön ligger i förhållande till Australien?

In [ ]:
#@title Kör detta dolda kodblock för att se var brandområdet ligger
import folium
m = folium.Map(
    location=[-35.7752,137.2142],
    tiles='Stamen Terrain',
    zoom_start=9
)

folium.Rectangle(
    bounds=[(-35.6,136.5), (-36.1,137)],
    fill=False
).add_to(m)


m

**Uppdrag 2B.b** Kör nedanstående kodblock för att ladda ner två satellitbilder. Den ena är en bild över området december 2019, och den andra januari 2020.

In [ ]:

!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/forest_fires/Australia/bands_2019_aus.npz --quiet
!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/forest_fires/Australia/bands_2020_aus.npz --quiet

#Banden för den australiska skogsbranden
bands19 = np.load('bands_2019_aus.npz')
bands20 = np.load('bands_2020_aus.npz')

#Plockar ut banden var för sig som float
#Färgbanden för december 2019
blue19=bands19["blue"].astype(float)
green19=bands19["green"].astype(float)
red19 = bands19["red"].astype(float)
nir19 = bands19["nir"].astype(float)

#Färgbanden för januari 2020
blue20=bands20["blue"].astype(float)
green20=bands20["green"].astype(float)
red20 = bands20["red"].astype(float)
nir20 = bands20["nir"].astype(float)

print("Klart!")


Till att börja med kan du undersöka färgbilder av området. För att skapa färgbilderna behöver du kombinera tre färgband som satelliten mäter. En vanlig färgbild är en kombination av rött, grönt och blått ljus.  Du kanske har lagt märke till att det finns ytterligare färgband i koden ovan som kallas `nir19` eller `nir20`. NIR är förkortning för Near InfraRed, vilket är det ljus som har lite längre våglängd än rött. Det går inte att se med blotta ögat men satelliten har instrument som kan plocka upp det. NIR kommer att behövas lite senare i uppgiften.

**Uppdrag 2B.c** Kör kodrutan nedan för att definiera funktionen `color_img` som skapar en färgbild av det röda, gröna och blåa bandet. 

In [ ]:
def color_img(red,green,blue):
  rgb=np.zeros([500,625,3],dtype=np.uint8)
  divide=8
  for x in range(500):
    for y in range(625):
      r = min(255, red[x][y]/divide)
      g = min(255, green[x][y]/divide)
      b = min(255, blue[x][y]/divide)
      rgb[x,y] = [r,g,b]
  return rgb

print("Klart!")

Funktionen skapar en bild på ett format som passar för plottning.

**Uppdrag 2B.d** Kör kodrutan nedan för att importera biblioteket PIL som vi ska använda för plottning.

In [ ]:
from PIL import Image

print("Klart!")

Nu ska vi plotta färgbilderna för 2019 och 2020 för att se om det är någon skillnad mellan
bilderna.

Uppdrag **2B.e** Kör kodrutan nedan för att plotta färgbilden för 2019. Kan du urskilja land och hav? Går det att se vad som är skog eller annan växtlighet?

In [ ]:
rgb=color_img(red19,green19,blue19)

img = Image.fromarray(rgb)
display(img)

**Uppdrag 2B.f** Modifiera koden i rutan nedan så att du i stället plottar färgbilden för 2020. Färgbanden ska vara i ordningen rött, grönt, blått. Ser du någon skillnad mot 2019? Var tror du att branden har härjat? 

In [ ]:
# Byt ut ??? så du får färgbilden för 2020
rgb=color_img(???,???,???)

img = Image.fromarray(rgb)
display(img)

<details>
<summary markdown="span">
Svar
</summary>
<p>

För att se färgbilden för december 2019:
<pre><code>rgb=color_img(red19,green19,blue19)</code></pre>
För att se färgbilden för januari 2020:
<pre><code>rgb=color_img(red20,green20,blue20)</code></pre>
</p>
</details>
<p></p>

I nästa steg ska vi skapa en bild som visar Normalized Difference Vegetation Index (NDVI). Förhoppningsvis har du stött på NDVI i tidigare uppgifter, som Torkan A.

Som du kanske kommer ihåg så är NDVI ett tal mellan -1 och 1, och som kan användas för att dela upp en landyta i olika kategorier. Ett index som är över 0.1 kan i de Eesta fall sägas vara någon form av vegetation. Ju högre NDVI desto tätare växtlighet. Ett NDVI-värde mellan -0.1 till 0.1 brukar indikera en landyta fri från vegetation. Negativa NDVI-värden mellan -0.1 till -1 brukar istället betyda att det är en yta täckt av vatten.

NDVI räknas ut med hjälp av NIR och det röda bandet enligt följande formel: 

           NIR - Red
    NDVI = ---------
           NIR + Red

NDVI-formeln utnyttjar att reflektionen skiljer sig mycket åt mellan NIR och rött ljus för Flera slags ytor. Växtlighet absorberar mycket rött ljus, men reflekterar NIR-ljus i hög grad. Med vatten förhåller det sig tvärtom.

**Uppgift 2B.g** Skriv en funktion med namn `ndvi_img` som räknar ut NDVI i följande kodblock. Om du är osäker på syntaxen får du ett färdigt kodskelett under tipsfliken. Du behöver dock själv skriva in vilka argument funktionen behöver och hur NDVI ska räknas ut. Om du har gjort funktioner innan så prova gärna själv innan du kikar på tipset. Var noga med placeringen av parenteser!

In [ ]:
#Skriv din funktion ndvi_img här:
def ...


print("Klart!") #Utskriften här är bara för att du ska se att den händer något när kodrutan körs.

<details>
<summary markdown="span">
Tips!
</summary>
<p><pre><code>def ndvi_img(???,???):
  #skriv hur ndvi räknas ut här:
  ndvi=
  return ndvi
</code></pre>
</p>
</details>
<p></p>

<details>
<summary markdown="span">
Lösning
</summary>
<p><pre><code>def ndvi_img(nir,red):
 ndvi=(nir-red)/(nir+red)
 return ndvi
</code></pre>
</p>
</details>
<p></p>

Nu kan du prova din funktion genom att se hur NDVI för området ser ut. Börja med år 2019 och fortsätt sedan med 2020. 

**Uppdrag 2B.h** Använd de två kodblocken nedan för att plotta NDVI för 2019 och 2020. För varje kodblock, skriv in rätt argument i anropet av din `ndvi_img` funktion och kör sedan kodblocket. Om du är osäker på namnen på färgbanden så kan du hitta dem i kodblocket som skapar färgbilderna.

In [ ]:
#Byt ut ??? mot namnen på rätt färgband:
ndvi19 = ndvi_img(???,???)

plt.figure(figsize=(10,10))
plt.pcolormesh(ndvi19, cmap='brg')
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
plt.ylim(ndvi19.shape[0], 0)
plt.clim(-1.0, 0.7)
plt.colorbar(label='NDVI')
plt.show()

In [ ]:
#Byt ut ??? mot namnen på rätt färgband:
ndvi20 = ndvi_img(???,???)

plt.figure(figsize=(10,10))
plt.pcolormesh(ndvi20, cmap='brg')
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
plt.ylim(ndvi20.shape[0], 0)
plt.clim(-1.0, 0.7)
plt.colorbar(label='NDVI')
plt.show()

<details>
<summary markdown="span">
Svar
</summary>
<p><pre><code> ndvi19=ndvi_img(nir19,red19)
 ndvi20=ndvi_img(nir20,red20)
</code></pre>
</p>
</details>
<p></p>

**Uppdrag 2B.i** Jämför de båda NDVI-bilderna. Kan du identifiera vad som är växtlighet respektive vatten? I vilka områden tror du skogsbranden har ägt rum?

**Uppdrag 2B.j** Scrolla upp och titta på färgbilderna igen. Är det lättare att se identifiera vad som är vad om man tittar på NDVI jämfört med färgbilder? Vad tror du orsakar de röda prickarna i vattnet? Scrolla upp och jämför med färgbilderna, går det att se något där?


<details>
<summary markdown="span">
Kommentar
</summary>
Det går mycket lättare att se vad som är växtlighet i NDVI-bilden än i färgbilden. De röda prickarna i havet på NDVI-bilden är dock uppenbart inte barmark. Det kan därför vara bra att titta både på färgbilderna och NDVI-bilderna för att avgöra vad som är vad.
</details>
<p></p>

Vi ska nu skapa en polygon för att uppskatta storleken på området som har brunnit.

**Uppgift 2B.k** Börja med att köra kodblocket nedan för att få upp en större bild av NDVI över området år 2020.


In [ ]:
plt.figure(figsize=(15,15))
plt.pcolormesh(ndvi20, cmap='brg')
plt.ylim(ndvi20.shape[0], 0)
plt.clim(-1.0, 0.7)
plt.colorbar(label='NDVI')
plt.xticks(np.arange(0, 625+1, 25.0))
plt.yticks(np.arange(0, 500+1, 25.0))
plt.grid()
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True 

#Bygg ut listan med fler hörn så att hela brandområdet täcks:
corners = [[540, 130],[525, 155],[510, 190],[540, 200],[570, 210],
           [590, 120],[620, 115],[620, 250],[580, 280],[550, 280],
           [510, 260],[500, 230],[475, 225],[460, 260],[420, 300],
           [460, 310],[460, 330],[425, 325],[450, 350],[475, 355]]

             
           
polygon = plt.Polygon(corners,fill=None)
plt.gca().add_patch(polygon)
plt.show()

Om du tittar i bilden ser du en polygon som påbörjats för att ringa in brandområdet. Hittar du polygonen i koden också?

**Uppdrag 2B.l** Lägg till 3-4 hörn i `corners` så att polygonen täcker området ungefärligt. Lägg sedan till fler hörn och anpassa polygonen så att den täcker brandområdet mer noggrannt. Du väljer själv hur noggrann du vill vara. Kör kodrutan efter varje ändring du gör, så du kontrollerar hur resultatet blir.

<details>
<summary markdown="span">
Tips
</summary>
Du kan behöva runt 30 hörn till för att få en riktigt noggrann täckning, som i följande bild. Men det räcker med mycket färre hörn för att få en ungefärlig uppskattning.<br>

<img src="https://github.com/lunduniversity/schoolprog-satellite/raw/master/exercises/forest_fires/AUS_firepolygons.png" width="800"/>
</details>
<p></p>

Nu ska vi räkna ut arean på polygonen. Vi kan använda funktionen `area_calc` som vi definerade tidigare. Parameter till `area_calc` är listan av koordinater. Fyll i rätt parametrar nedan och räkna ut arean.

**Uppdrag 2B.m** Skriv in rätt parametervärde till area_calc för att räkna ut arean i kodblocket nedan.

In [ ]:
#Byt ut ??? mot rätt parametrar
totalarea =area_calc(???)

print(totalarea, 'pixlar')

<details>
<summary markdown="span">
Svar
</summary>
<p><pre><code>totalarea=area_calc(corners)</code></pre>
Arean borde vara någonstans mellan 130 000 - 150 000 pixlar.
</p>
</details>
<p></p>

Nu har vi räknat ut arean i pixlar. Men hur mycket kan det motsvara i $km^2$ (kvadratkilometer)?

Koordinataxlarna i bilden är graderad i pixlar. En pixel i bilden motsvarar 14400 $m^2$ (kvadratmeter). Om du är osäker på omvandlingen kan du börja med att räkna ut hur många $m^2$ brandområdet motsvarar.

**Uppdrag 2B.n** Räkna ut arean i kvadratmeter i kodblocket nedan.


In [ ]:
# Byt ut ??? för att räkna ut vad arean är i kvadratmeter.
area_m2 = ???
print(area_m2,'m^2')

<details>
<summary markdown="span">
Lösning
</summary>
<p><pre><code> area_m2=totalarea*14400
</code></pre>
</p>
</details>
<p></p>

**Uppdrag 2B.m** Sen behöver du fundera ut hur arean kan omvandlas till $km^2$. Skriv in din omvandling i kodblocket nedan. 

In [ ]:
# Byt ut ??? för att räkna ut arean i kvadratkilometer
area_km2 =
print(area_km2, 'km^2')

<details>
<summary markdown="span">
Tips!
</summary>
<p>Som du säkert redan vet är 1000 meter samma som 1 km. 1 $km^2$ är då samma som $1000^2 m^2$, alltså $1000000 m^2$.
</p>
</details>
<p></p>

<details>
<summary markdown="span">
Lösning
</summary>
<p><pre><code> area_km2=area_m2/1000000
</code></pre>
</p>
</details>
<p></p>

Skog räknar man ofta i *hektar*, där en hektar är 10000 $m^2$, eller 0.01 $km^2$. Multiplicera alltså ditt resultat med 100 för att få det antal hektar som brann på Kangaroo Island. Ett annat mått på ytor är "fotbollsplaner". En fotbollsplan är ofta 60 x 90 meter, dvs det går ungefär 2 fotbollsplaner på en hektar.


**Uppgift 2A.p** Hur många hektar resp. fotbollsplaner motsvarar branden på Kangaroo Island enligt din beräkning? Räkna gärna ut det genom att programmera i kodrutan nedan. Tror du att detta var mycket eller lite skog i relation till andra skogsbränder? Vad händer i samhället när det blir stora bränder? Kan vi göra något för att förhindra skogsbränder i framtiden? Diskutera gärna med en kompis.


In [ ]:
# Räkna ut hur många hektar som brann och hur många fotbollsplaner det motsvarar


<details>
<summary markdown="span">
Kommentar
</summary>
Bränderna i Australien var väldigt stora. Man räknar med att det brann mer än 18 miljoner hektar under den australiensiska sommaren 2019-2020. Som jämförelse brann det 25000 hektar skog i Sverige under den torra sommaren 2018, vilket ändå räknas som väldigt stora skador i Sverige.<br><br>
Det finns stora glest befolkade områden med buskvegetation i Australien. Bränderna var så stora att om vi hade tittat på fastlandet i stället för på en ö så hade vi fått använda flera satellitbilder. När det blir så stora bränder får man fokusera på att skydda människor och försöka hindra elden från att nå större tätorter.<br><br>
Massor av djur dör naturligtvis när det är skogsbrand. På Kangaroo Island fanns många hotade arter, både däggdjur, fåglar, och spindlar. Man tror att några av spindelarterna dött ut i branden, och framtiden är osäker för flera fågelarter och arter av koala.<br><br>
Du kan läsa mer om bränderna i Australien 2019-2020 på t.ex. <a href="https://en.wikipedia.org/wiki/2019–20_Australian_bushfire_season">denna wikipedia-artikel</a>.
</details>
<p></p>

## 3. Quiz

Kör kodrutan nedan för att starta quizzet. Kommer du ihåg vad vi har gått igenom?

In [ ]:
#@title Starta quizzet!
!wget https://raw.githubusercontent.com/lunduniversity/schoolprog-satellite/master/for_developers/quiz.py --quiet
!wget https://raw.githubusercontent.com/lunduniversity/schoolprog-satellite/master/exercises/forest_fires/quiz_skogsbrand.json --quiet
import json
import quiz

with open("quiz_skogsbrand.json") as f:
  quiz_dict = json.load(f)
quiz.main(quiz_dict)

<hr size=5 noshade>
<small>
Denna uppgift ingår i <a href="https://lunduniversity.github.io/schoolprog-satellite/">LU Miljödataprogrammering</a>. Copyright © 2020 Lunds universitet, ICOS Carbon Portal och Svenska Science Centers. License: <a href="http://creativecommons.org/licenses/by/4.0/">CC BY 4.0</a>
</small>